In [79]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import os
import cv2

In [80]:
ls

 Volume in drive D is New Volume
 Volume Serial Number is 2E70-070D

 Directory of D:\University\Y5 S1\COMP5425\Project\DATA\seborrheic_keratosis

29/04/2021  08:46 PM    <DIR>          .
29/04/2021  08:46 PM    <DIR>          ..
29/04/2021  10:03 PM    <DIR>          imgs
29/04/2021  08:43 PM           146,765 metadata.csv
               1 File(s)        146,765 bytes
               3 Dir(s)  1,898,764,197,888 bytes free


In [81]:
cd "DATA"

[WinError 2] The system cannot find the file specified: 'DATA'
D:\University\Y5 S1\COMP5425\Project\DATA\seborrheic_keratosis


In [82]:
cd "seborrheic_keratosis"

[WinError 2] The system cannot find the file specified: 'seborrheic_keratosis'
D:\University\Y5 S1\COMP5425\Project\DATA\seborrheic_keratosis


## Loading the metadata

In [83]:
# Open csv file
import pandas as pd 
data = pd.read_csv("metadata.csv", index_col="name") # Index on image name

In [84]:
data.head()

,_id,meta.clinical.age_approx,meta.clinical.anatom_site_general,meta.clinical.benign_malignant,meta.clinical.clin_size_long_diam_mm,meta.clinical.diagnosis,meta.clinical.diagnosis_confirm_type,meta.clinical.family_hx_mm,meta.clinical.lesion_id,meta.clinical.mel_class,...,meta.clinical.sex,meta.acquisition.acquisition_day,meta.acquisition.blurry,meta.acquisition.color_tint,meta.acquisition.dermoscopic_type,meta.acquisition.hairy,meta.acquisition.image_type,meta.acquisition.marker_pen,meta.acquisition.pixelsX,meta.acquisition.pixelsY
name,,,,,,,,,,,,,,,,,,,,,
ISIC_0001104,54e755f4bae47850e86cdfdb,75.0,posterior torso,benign,NaN,seborrheic keratosis,histopathology,NaN,NaN,NaN,...,male,NaN,NaN,NaN,NaN,NaN,dermoscopic,NaN,3072,2304
ISIC_0001113,54e755f8bae47850e86ce008,80.0,head/neck,benign,NaN,seborrheic keratosis,histopathology,NaN,NaN,NaN,...,male,NaN,NaN,NaN,NaN,NaN,dermoscopic,NaN,3008,2000
ISIC_0001123,54e755fbbae47850e86ce03a,70.0,head/neck,benign,NaN,seborrheic keratosis,histopathology,NaN,NaN,NaN,...,male,NaN,NaN,NaN,NaN,NaN,dermoscopic,NaN,1936,1936
ISIC_0001144,54e75602bae47850e86ce0a3,60.0,anterior torso,benign,NaN,seborrheic keratosis,histopathology,NaN,NaN,NaN,...,male,NaN,NaN,NaN,NaN,NaN,dermoscopic,NaN,3072,2304
ISIC_0001145,54e75602bae47850e86ce0a8,80.0,lower extremity,benign,NaN,seborrheic keratosis,histopathology,NaN,NaN,NaN,...,female,NaN,NaN,NaN,NaN,NaN,dermoscopic,NaN,3072,2304


The most relevant attributes are:
* **name** = the filename of the image this metadata belongs to
* **meta.clinical.age_approx** = approximate age of the patient (<40/40-60/60+)
* **meta.clinical.anatom_site_general** = anatomic site of the lesion (anterior torso/head or neck/upper extremity/lower extremity/other)
* **meta.clinical.sex** = gender (male/female)

In [85]:
# Select the relevant headers
data = data[["meta.clinical.age_approx", "meta.clinical.anatom_site_general", "meta.clinical.sex"]]

In [86]:
data.head()

,meta.clinical.age_approx,meta.clinical.anatom_site_general,meta.clinical.sex
name,,,
ISIC_0001104,75.0,posterior torso,male
ISIC_0001113,80.0,head/neck,male
ISIC_0001123,70.0,head/neck,male
ISIC_0001144,60.0,anterior torso,male
ISIC_0001145,80.0,lower extremity,female


## Loading the images

In [87]:
# Directory of the images and metadata
root_dir = "imgs/"

# Seborrheic Keratosis
skin_class = "2"

# Class AGE
def getAge(age):
    
    if np.isnan(age): # Unknown
        return "0"
    
    else:
        age=int(age)
        if age < 40: # <40
            return "1"
        elif age >=40 and age <60: # 40-60
            return "2"
        else: # 60+
            return "3"

# Class GENDER
def getGender(gender):
    if gender == "nan": # Unknown
        return "0"
    elif gender == "male": 
        return "1"
    else: 
        return "2"

# Class ANATOMIC SITE
def getSite(site):
    if site == "anterior torso": 
        return "1"
    elif site == "head/neck":
        return "2"
    elif site == "lower extremity":
        return "3"
    elif site == "upper extremity":
        return "4"
    else: # Other locations/Unknown
        return "5"


# Iterate ALL images in directory

for filename in glob.iglob(root_dir + '**/*.jpg', recursive=True):

    imgName = filename.replace("imgs\\","")
    imgName = imgName.replace(".jpg", "")
    
    # Get attributes
    age = data.loc[imgName]["meta.clinical.age_approx"]
    gender = str(data.loc[imgName]["meta.clinical.sex"])
    site = str(data.loc[imgName]["meta.clinical.anatom_site_general"])
    
    age = getAge(age)
    gender = getGender(gender)
    site = getSite(site)
    
    # Rename image
    newName = skin_class + "_" + gender + "_" + age + "_" + site + "_" + imgName
    print(newName)
    
    #### Display image for testing ####
#     image = cv2.imread(filename)
#     plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
#     break
    ###################################
    
    os.rename(r'' + filename,r'imgs/' + newName + ".jpeg")

2_1_3_5_ISIC_0001104
2_1_3_2_ISIC_0001113
2_1_3_2_ISIC_0001123
2_1_3_1_ISIC_0001144
2_2_3_3_ISIC_0001145
2_1_3_4_ISIC_0001156
2_1_3_2_ISIC_0010389
2_2_3_2_ISIC_0010491
2_2_2_4_ISIC_0010985
2_1_1_5_ISIC_0012086
2_2_3_5_ISIC_0012090
2_1_3_5_ISIC_0012103
2_1_3_5_ISIC_0012117
2_2_3_5_ISIC_0012134
2_1_3_5_ISIC_0012136
2_1_3_5_ISIC_0012143
2_1_3_5_ISIC_0012155
2_1_3_5_ISIC_0012170
2_2_3_5_ISIC_0012172
2_2_2_5_ISIC_0012173
2_1_2_5_ISIC_0012178
2_2_2_5_ISIC_0012183
2_1_3_5_ISIC_0012199
2_1_3_5_ISIC_0012204
2_1_3_5_ISIC_0012207
2_1_2_5_ISIC_0012210
2_2_2_5_ISIC_0012214
2_2_3_5_ISIC_0012215
2_1_3_5_ISIC_0012217
2_2_2_5_ISIC_0012223
2_2_3_5_ISIC_0012224
2_2_3_5_ISIC_0012227
2_2_2_5_ISIC_0012237
2_2_2_5_ISIC_0012238
2_2_3_5_ISIC_0012240
2_1_3_5_ISIC_0012245
2_1_3_5_ISIC_0012248
2_1_3_5_ISIC_0012254
2_2_3_5_ISIC_0012265
2_1_3_5_ISIC_0012266
2_2_2_5_ISIC_0012272
2_1_3_5_ISIC_0012273
2_1_3_5_ISIC_0012282
2_2_3_5_ISIC_0012291
2_1_2_5_ISIC_0012294
2_1_3_5_ISIC_0012299
2_1_3_5_ISIC_0012314
2_1_3_5_ISIC_

2_2_3_2_ISIC_0055795
2_1_3_3_ISIC_0055800
2_1_3_3_ISIC_0055816
2_1_3_1_ISIC_0055839
2_2_1_2_ISIC_0055894
2_1_3_1_ISIC_0055942
2_2_3_3_ISIC_0055955
2_1_2_1_ISIC_0055986
2_2_3_4_ISIC_0055988
2_2_3_3_ISIC_0056045
2_2_3_5_ISIC_0056053
2_1_3_4_ISIC_0056094
2_1_3_2_ISIC_0056107
2_1_3_1_ISIC_0056132
2_1_3_1_ISIC_0056149
2_2_3_1_ISIC_0056174
2_2_2_2_ISIC_0056175
2_2_3_1_ISIC_0056193
2_1_3_2_ISIC_0056231
2_2_3_3_ISIC_0056240
2_1_3_1_ISIC_0056241
2_2_2_2_ISIC_0056245
2_2_3_3_ISIC_0056312
2_1_3_2_ISIC_0056329
2_1_3_1_ISIC_0056416
2_1_3_4_ISIC_0056431
2_2_3_2_ISIC_0056432
2_1_3_2_ISIC_0056445
2_2_3_2_ISIC_0056456
2_2_1_4_ISIC_0056499
2_1_2_1_ISIC_0056514
2_1_1_1_ISIC_0056515
2_2_2_2_ISIC_0056517
2_1_2_1_ISIC_0056519
2_2_3_3_ISIC_0056521
2_1_3_1_ISIC_0056530
2_1_2_1_ISIC_0056535
2_1_2_3_ISIC_0056564
2_1_2_2_ISIC_0056586
2_2_3_1_ISIC_0056620
2_1_3_3_ISIC_0056622
2_2_3_2_ISIC_0056650
2_1_2_2_ISIC_0056654
2_1_2_2_ISIC_0056672
2_1_2_2_ISIC_0056687
2_1_2_1_ISIC_0056693
2_1_3_5_ISIC_0056699
2_2_3_2_ISIC_